In [33]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
from gensim import corpora, models
import pickle;
import pyLDAvis
import pyLDAvis.gensim

%matplotlib inline

/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [5]:
import csv
import re
#import spacy
import sys
import requests
import pandas as pd
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
import os
import sys, getopt
import numpy as np
from bs4 import BeautifulSoup
import urllib3
from glob import glob
from string import punctuation

from nltk.corpus import wordnet

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def find_pdfs():
    return glob(os.path.join('./AllSyllabiParser',"*.{}".format('pdf')))

def convert(fname, pages=None):
    """ Function converting pdf to string """
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    
    try:
        for page in PDFPage.get_pages(infile, pagenums):
            interpreter.process_page(page)
    except PDFTextExtractionNotAllowed:
        print('This pdf won\'t allow text extraction!')
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return strip_punctuation(text)

def word_count(string):
    my_string = string.lower().split()
    my_dict = {}
    for item in my_string:
        if item in my_dict:
            my_dict[item] += 1
        else:
            my_dict[item] = 1
    


In [6]:
list_of_pdfs = find_pdfs()
syllabus_string = ''
bag_of_words_all_syllabi = []
corpus = []
#Converting pdf to string
for pdf in list_of_pdfs:
    syllabus_string = convert(pdf)
    corpus.append(syllabus_string)
#     bag_of_words_one_syllabus = word_count(syllabus_string)
#     bag_of_words_all_syllabi.append(bag_of_words_one_syllabus)

This pdf won't allow text extraction!
This pdf won't allow text extraction!


In [11]:
# clean text

from nltk.corpus import words
from nltk.corpus import stopwords


allEnglishWords = set(words.words())
stopwordsList = set(stopwords.words('english'))

def clean_doc(doc, english = False, return_list = False, stopwords = False):
    listOfWords = re.findall(r'[^\d\W]{2,}', doc)
    listOfWords = [word.lower() for word in listOfWords]
    if english:
        listOfWords = [word for word in listOfWords if word in allEnglishWords]
    if stopwords:
        listOfWords = [word for word in listOfWords if word not in stopwordsList]
    if return_list:
        return listOfWords
    else:
        return " ".join(listOfWords)


In [13]:
cleaned_corpus = [clean_doc(doc, return_list = True, stopwords = True) for doc in corpus]

In [14]:
cleaned_corpus_1 = [clean_doc(doc, english = True, return_list = True, stopwords = True) for doc in corpus]

In [15]:
cleaned_corpus_2 = [clean_doc(doc, english = True, return_list = False, stopwords = True) for doc in corpus]

In [16]:
cleaned_corpus_3 = [clean_doc(doc, english = False, return_list = False, stopwords = True) for doc in corpus]

In [17]:
cleaned_corpus_4 = [clean_doc(doc, english = False, return_list = True, stopwords = True) for doc in corpus]

In [18]:
vectorizer = CountVectorizer(analyzer='word', stop_words='english', lowercase=True, token_pattern=r'[^\d\W]{2,}');
x_counts = vectorizer.fit_transform(cleaned_corpus_2);

In [19]:
words = vectorizer.get_feature_names()
words

['aa',
 'abdomen',
 'abduction',
 'aberration',
 'abide',
 'ability',
 'able',
 'abnormal',
 'abreast',
 'abroad',
 'absence',
 'absent',
 'absenteeism',
 'absolute',
 'absolutely',
 'absorb',
 'absorption',
 'abstract',
 'abstracted',
 'abstraction',
 'abstractly',
 'abundance',
 'abundant',
 'abuse',
 'academic',
 'academically',
 'academy',
 'accelerate',
 'accelerated',
 'acceleration',
 'accelerometer',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'acceptor',
 'access',
 'accessibility',
 'accessible',
 'accessory',
 'accident',
 'accidental',
 'accommodate',
 'accommodation',
 'accompany',
 'accomplish',
 'accomplished',
 'accordance',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accounting',
 'accreditation',
 'accrual',
 'accrue',
 'accumulate',
 'accumulation',
 'accuracy',
 'accurate',
 'accurately',
 'accustomed',
 'acetylation',
 'achieve',
 'achievement',
 'acid',
 'acidification',
 'acoustical',
 'acquaint',
 'acquainted',
 'acq

In [20]:
id2word = {}
for i, word in enumerate(words):
    id2word[i] = word

In [21]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [22]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [23]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [24]:
num_topics = 25
model = NMF(n_components=num_topics, init='nndsvd')
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=25, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [25]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25
0,class,lesson,language,lab,art,history,physics,trigonometric,lesson,design,...,energy,space,inflation,architectural,geography,media,psychology,health,calculus,electricity
1,student,days,culture,lecture,works,historical,magnetism,ti,medical,engineering,...,lesson,flight,economic,architecture,human,video,psychological,justice,derivative,lesson
2,course,activity,presentational,biology,studio,question,electric,trigonometry,body,project,...,sustainable,lesson,aggregate,lesson,exam,audio,psych,public,graph,electronics
3,chapter,project,interpersonal,laboratory,artistic,argument,electricity,polar,disease,element,...,water,aeronautics,supply,design,statistics,web,quiz,social,differential,digital
4,grade,create,interpretive,exam,portfolio,explain,course,angle,patient,process,...,alternative,aircraft,demand,green,population,converting,chapter,course,calculator,circuit
5,instructor,unit,communication,organic,dimensional,evidence,motion,exponential,human,solution,...,impact,moon,economics,sustainable,score,element,complete,oer,algebraic,magic
6,assignment,computer,audio,genetics,drawing,relative,laboratory,triangle,investigate,modeling,...,lead,design,unemployment,commercial,placement,course,behavior,socioeconomic,function,design
7,exam,lead,cultural,ecology,ce,significance,scientific,algebra,murder,lesson,...,renewable,travel,money,ga,environmental,use,abnormal,unequal,logarithmic,trip
8,canvas,science,print,course,work,course,mechanics,logarithmic,illness,lead,...,way,propulsion,market,shipping,course,delivery,research,status,exponential,basic
9,week,way,respond,anatomy,content,world,introductory,inverse,crime,unit,...,thermostat,traveling,fiscal,construction,advanced,linking,course,class,theorem,electronic


**LDA**

In [26]:
id2word = gensim.corpora.Dictionary(cleaned_corpus_1);
id2word

In [27]:
bow_corpus = [id2word.doc2bow(doc) for doc in cleaned_corpus_1]

In [28]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [29]:
lda = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=20, id2word=id2word, passes=50)

In [30]:
lda_tfidf = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=20, id2word=id2word, passes=50)

In [31]:
lda_tfidf.show_topics()

[(19,
  '0.003*"architectural" + 0.002*"certificate" + 0.002*"studio" + 0.002*"trial" + 0.002*"achievement" + 0.002*"announcement" + 0.001*"vex" + 0.001*"robot" + 0.001*"sustainable" + 0.001*"shell"'),
 (1,
  '0.003*"force" + 0.003*"asp" + 0.002*"light" + 0.002*"looping" + 0.002*"wave" + 0.002*"acceleration" + 0.002*"newton" + 0.002*"momentum" + 0.002*"diffraction" + 0.002*"speed"'),
 (14,
  '0.006*"lesson" + 0.004*"class" + 0.004*"lab" + 0.004*"design" + 0.004*"chapter" + 0.003*"instructor" + 0.003*"laboratory" + 0.003*"lecture" + 0.003*"student" + 0.003*"art"'),
 (17,
  '0.005*"culture" + 0.005*"presentational" + 0.005*"interpersonal" + 0.005*"language" + 0.004*"interpretive" + 0.004*"audio" + 0.003*"print" + 0.003*"literature" + 0.003*"post" + 0.003*"cellular"'),
 (16,
  '0.003*"marketing" + 0.002*"statistics" + 0.002*"electrochemistry" + 0.001*"aircraft" + 0.001*"redox" + 0.001*"equilibria" + 0.001*"works" + 0.001*"nuclear" + 0.001*"dogs" + 0.001*"ionic"'),
 (11,
  '0.002*"ocean" +

In [40]:
vis_data =pyLDAvis.gensim.prepare(lda_tfidf, bow_corpus, id2word)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [41]:
pyLDAvis.display(vis_data)

In [36]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [23]:
get_lda_topics(lda, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,due,course,student,lesson,class,class,lab,design,physics,syllabus,design,canvas,course,class,system,course,class,course,days,class
1,complete,academic,class,design,discussion,student,lecture,engineering,energy,waymaker,lesson,physical,student,student,course,exam,read,class,activity,course
2,quiz,student,course,body,please,course,course,project,course,demand,building,excel,may,course,anatomy,history,reading,college,lesson,computer
3,psychology,policy,instructor,project,speech,grade,may,unit,chapter,final,commercial,assistance,class,work,exam,college,economic,exam,project,certificate
4,course,disability,policy,unit,course,may,exam,course,experiment,topic,project,textbook,design,week,class,historical,writing,language,create,use
5,chapter,instructor,may,explore,student,learning,laboratory,process,laboratory,price,engineering,via,instructor,may,human,score,homework,student,data,security
6,student,learning,attendance,way,textbook,canvas,class,problem,motion,module,architecture,computer,use,grade,describe,question,week,learning,problem,history
7,psychological,work,sexual,lead,work,exam,biology,use,law,supply,architectural,shall,lesson,instructor,body,placement,international,information,computer,learn
8,psych,state,academic,investigate,week,day,student,lesson,understanding,class,development,financial,lab,final,lab,program,assignment,use,page,login
9,discussion,must,office,course,communication,must,cell,way,electric,section,unit,module,system,exam,student,world,inflation,instructor,course,week


In [37]:
get_lda_topics(lda_tfidf, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,health,force,speech,duiker,disease,political,comparative,inflation,anatomy,slope,audio,ocean,calibration,proportion,lesson,eh,marketing,culture,ing,architectural
1,justice,asp,criminal,march,human,accounting,cancer,aggregate,human,arithmetical,rhetorical,matrix,spectrophotometry,hybrid,class,deep,statistics,presentational,statistics,certificate
2,waymaker,light,preface,weight,patient,politics,toxin,money,flight,prototype,thy,music,extraction,mail,lab,sustainable,electrochemistry,interpersonal,sinusoidal,studio
3,sampling,looping,button,movie,anthropology,health,machine,unemployment,nervous,equation,nonfiction,beaker,chromatography,active,design,patriot,aircraft,language,expenditure,trial
4,probability,wave,crime,civilization,anna,worth,dysfunction,monetary,animal,whole,argumentative,western,iris,shorter,chapter,duel,redox,interpretive,forcing,achievement
5,carboxylic,acceleration,war,capitalization,illness,government,infection,fiscal,anatomical,perimeter,rotation,vector,speech,calculated,instructor,thermostat,equilibria,audio,voltage,announcement
6,ba,newton,pause,pig,logic,end,diagnose,studio,dissection,subtraction,sole,marine,instrumentation,platform,laboratory,war,works,print,algae,vex
7,status,momentum,justice,folder,mystery,west,smith,electronics,car,capstone,peer,derivative,precision,consecutive,lecture,turning,nuclear,literature,renewable,robot
8,nomenclature,diffraction,torture,punctuation,duration,mon,vex,nuclear,muscular,resemble,passage,museum,solvent,justification,student,wages,dogs,post,aa,sustainable
9,negative,speed,chief,toy,murder,heart,mike,banking,organ,adaptively,lens,theory,repeat,color,art,cold,ionic,cellular,capacitance,shell


In [54]:
get_lda_topics(lda, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,course,course,inserted,music,course,course,course,accounting,design,course,body,class,class,course,student,government,lesson,course,lab,economic
1,exam,health,hypothetical,redox,student,class,laboratory,class,lesson,text,lesson,speech,course,math,class,political,days,class,lecture,read
2,language,class,shark,equilibria,class,student,may,chapter,engineering,reading,human,course,due,test,course,politics,project,student,biology,policy
3,college,syllabus,dioxide,colligative,learning,use,academic,student,project,language,history,student,history,instructor,instructor,us,activity,exam,exam,course
4,culture,grade,crude,ionic,may,may,chapter,course,unit,literature,investigate,chapter,chapter,linear,policy,course,design,instructor,cell,textbook
5,score,student,plate,electrochemistry,physics,exam,lab,exam,energy,exam,disease,instructor,complete,trigonometric,may,exam,create,may,cellular,supply
6,communication,instructor,culmination,process,must,language,must,instructor,way,communication,medical,use,reading,mathematics,attendance,one,course,college,may,class
7,advanced,assignment,downstream,worth,work,lesson,policy,financial,process,culture,course,may,quiz,ti,participation,explain,data,system,course,demand
8,placement,public,humanity,dissolution,complete,lab,student,subject,use,literary,explore,college,student,solve,academic,system,unit,week,life,sexual
9,program,social,cooking,scales,college,college,learning,work,lead,study,historical,learning,paper,disability,time,united,way,learning,laboratory,discussion


In [74]:
get_lda_topics(lda_tfidf, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,lesson,war,comparative,marketing,psychology,lab,physics,lesson,health,inflation
1,speech,ecology,trial,cancer,psych,design,presentational,duiker,lesson,aggregate
2,game,revolution,fieldwork,ba,asp,chapter,motion,architectural,body,unemployment
3,python,religion,wearable,rhythm,lesson,class,lesson,diffraction,skeletal,lesson
4,mobile,poem,intern,ecology,criminal,laboratory,interpretive,architecture,anatomy,supply
5,psychology,fungi,market,musical,infant,lecture,culture,nuclear,justice,money
6,movie,wartime,demand,fungi,child,instructor,electricity,commercial,matrix,vex
7,inventor,whitehead,robot,dogs,heart,student,electric,audio,waymaker,monetary
8,installation,billion,execute,entrepreneurship,provider,project,interpersonal,prof,flight,fiscal
9,pro,despite,conductive,joint,psychological,sexual,audio,radioactive,eh,anna


In [12]:
import lda
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cvectorizer = CountVectorizer(min_df=5, stop_words='english')
cvz = cvectorizer.fit_transform(cleaned_corpus_3)

In [14]:
n_topics = 20
n_iter = 20



lda_m = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_m.fit_transform(cvz)

INFO:lda:n_documents: 156
INFO:lda:vocab_size: 2832
INFO:lda:n_words: 125458
INFO:lda:n_topics: 20
INFO:lda:n_iter: 20
C:\Users\Public\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -1399328
INFO:lda:<10> log likelihood: -969903
INFO:lda:<19> log likelihood: -932084


In [15]:
lda_m.topic_word_

array([[2.09103531e-06, 1.33847170e-02, 2.51133341e-03, ...,
        2.09103531e-06, 2.09103531e-06, 2.09103531e-06],
       [1.73390760e-06, 1.73390760e-06, 2.42920455e-03, ...,
        1.73390760e-06, 1.73390760e-06, 1.73390760e-06],
       [1.91632556e-06, 1.91632556e-06, 1.91632556e-06, ...,
        1.91632556e-06, 1.91632556e-06, 1.91632556e-06],
       ...,
       [1.29595455e-06, 1.29595455e-06, 1.29595455e-06, ...,
        1.29595455e-06, 1.29595455e-06, 1.29595455e-06],
       [1.89669823e-06, 1.89669823e-06, 1.89669823e-06, ...,
        1.89669823e-06, 1.89669823e-06, 1.89669823e-06],
       [1.64871763e-06, 1.64871763e-06, 1.64871763e-06, ...,
        1.64871763e-06, 1.65036635e-03, 3.13421221e-03]])

In [17]:
vocab

In [16]:
 # model.components_ also works
vocab = gensim.corpora.Dictionary(cleaned_corpus_4)    
    
topic_word = lda_m.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(10958 unique tokens: ['academic', 'adaptively', 'addressed', 'advice', 'allowed']...) from 156 documents (total 153738 corpus positions)


TypeError: sequence item 0: expected str instance, numpy.int32 found